In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This notebook contains KMeans clustering for mall customer segmentation. Please give me suggestions to improve this notebook :)

# Inspecting Dataset

In [ ]:
df = pd.read_csv("/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv")

In [ ]:
df

In [ ]:
df.info()

The data is already clean, we don't have to do further preprocessing

In [ ]:
df.describe()

Let's inspect the distribution of each features

In [ ]:
sns.countplot(df.Gender)

I think female might have higher spending score

In [ ]:
print(df[df.Gender=="Female"]["Spending Score (1-100)"].mean())
print(df[df.Gender=="Male"]["Spending Score (1-100)"].mean())

well yes, but not that much

how about the distribution of other features?

In [ ]:
sns.distplot(df.Age, kde = False)

In [ ]:
sns.distplot(df["Annual Income (k$)"], kde = False)

In [ ]:
sns.distplot(df["Spending Score (1-100)"], kde = False)

ummm.. can't tell much,

let's plot spending score with annual income and age

In [ ]:
sns.scatterplot(x = df["Annual Income (k$)"], y =df["Spending Score (1-100)"], hue=df.Gender)

In [ ]:
sns.scatterplot(x = df.Age, y =df["Spending Score (1-100)"], hue=df.Gender)

In [ ]:
sns.pairplot(df)

It seems that we can do some clustering to divide the customer types, so we can focus on which customer to be the sales target.

# KMeans Clustering

I will divide clusters from two plot above, I will use elbow method to determine how many clusters I will use.

First, let's divide the customers based on their **income and spending**

In [ ]:
income_spending = df.iloc[:,3:5]
income_spending

In [ ]:
from sklearn.cluster import KMeans

wcss = []
for i in range(1, 11):
    km = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    km.fit(income_spending)
    wcss.append(km.inertia_)
    
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method', fontsize = 20)
plt.xlabel('No. of Clusters')
plt.ylabel('wcss')
plt.show()

It seems that five is the best number

In [ ]:
km = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
income_spending["cluster"] = km.fit_predict(income_spending)
income_spending

Labelling clusters

In [ ]:
cust_dict = {0:"target", 1:"usual", 2:"ideal", 3:"extravagant", 4:"possible"}
income_spending.cluster = income_spending["cluster"].map(cust_dict)

Plotting the clusters

In [ ]:
sns.scatterplot(x = "Annual Income (k$)", y = "Spending Score (1-100)", hue = "cluster", data = income_spending, palette="tab10")
plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1], s = 50, c = 'blue' , label = 'centeroid')
plt.legend(bbox_to_anchor=(1, 1.05))

Now, I will divide the customers based on their Age and Spending

In [ ]:
age_spending = df[["Age", "Spending Score (1-100)"]]

In [ ]:
age_spending

In [ ]:
wcss = []
for i in range(1, 11):
    km = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    km.fit(age_spending)
    wcss.append(km.inertia_)
    
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method', fontsize = 20)
plt.xlabel('No. of Clusters')
plt.ylabel('wcss')
plt.show()

Four is the best number

In [ ]:
km = KMeans(n_clusters = 4, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)

age_spending["cluster"] =  km.fit_predict(age_spending)
cust_dict = {0:"target (young)", 1:"ideal", 2:"potential", 3:"target(old)"}
age_spending["cluster"] = age_spending["cluster"].map(cust_dict)

age_spending

In [ ]:
sns.scatterplot(x="Age", y="Spending Score (1-100)", hue="cluster", data=age_spending, palette="tab10")
plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1], s = 50, c = 'blue' , label = 'centeroid')
plt.legend(bbox_to_anchor=(1, 1))

How about combining all features?

In [ ]:
df3d = df[["Age", "Annual Income (k$)", "Spending Score (1-100)"]]

In [ ]:
km = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
df3d["cluster"] = km.fit_predict(df3d)
cust_dict = {0:"ideal", 1:"possible", 2:"usual", 3:"extravagant", 4:"target"}
df3d.cluster = df3d.cluster.map(cust_dict)
df3d

In [ ]:
import plotly.graph_objs as go

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df3d, x="Annual Income (k$)", y="Spending Score (1-100)", z="Age",
                    color='cluster', title="Mall Customer Segmentation")

fig.show()

**That's it!**, please give me your suggestions to improve this notebook :)